In [38]:
import os

In [39]:
%pwd

'c:\\Users\\vprat\\Desktop\\portfolio\\Image-classification-Deep-Learning-Project'

In [19]:
os.chdir("Image-classification-Deep-Learning-Project")

In [20]:
%pwd

'c:\\Users\\vprat\\Desktop\\portfolio\\Image-classification-Deep-Learning-Project'

In [40]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [41]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml,create_directories



In [42]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [43]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size
import boto3

In [44]:
def download_file_from_s3(bucket_name, s3_file_key, local_file_path):
    s3 = boto3.client('s3')

    try:
        # Download the file from S3
        s3.download_file(bucket_name, s3_file_key, local_file_path)
        print(f"File {s3_file_key} downloaded from bucket {bucket_name} to {local_file_path}.")
    except Exception as e:
        print(f"Error downloading file from S3: {e}")

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            if dataset_url.startswith("s3://"):
                dataset_url = dataset_url[5:]
    
            # Split the URL into bucket name and file key
            bucket_name, s3_file_key = dataset_url.split('/', 1)
            print(bucket_name)
            print(s3_file_key)
            print("==============")
            # prefix = 'https://drive.google.com/uc?/export=download&id='
            download_file_from_s3(bucket_name, s3_file_key, zip_download_dir)
            #local_file_path = os.path.join(os.getcwd(), 'kidney-data-scan-images.zip')
            #gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [45]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 's3://image-dataset-mlops/kidney-image/kidney_scan_images.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}
[2024-06-30 03:32:36,138: INFO: common: yaml file: config\config.yaml loaded successfully {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 's3://image-dataset-mlops/kidney-image/kidney_scan_images.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}]
{'key': 'val'}
[2024-06-30 03:32:36,139: INFO: common: yaml file: params.yaml loaded successfully {'key': 'val'}]
[2024-06-30 03:32:36,140: INFO: common: created directory at: artifacts]
[2024-06-30 03:32:36,141: INFO: common: created directory at: artifacts/data_ingestion]
[2024-06-30 03:32:36,141: INFO: 1431902254: Downloading data from s3://image-dataset-mlops/kidney-im